## Spotify Data Ingestion

In this notebook we process CSV files and create a property graph.

In [1]:
# required libraries
import pandas as pd
import os
from pathlib import Path
import datetime


In [2]:
# Get absolute path
absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "datasets")
rdfPath = os.path.join(absPath, "rdf")

# Create dataset directory if not exists
if not os.path.exists(datasetsPath):
    os.mkdir(datasetsPath)

# Create RDF directory if not exists
if not os.path.exists(rdfPath):
    os.mkdir(rdfPath)

# Setup datasets paths
spotifyChartsPath = os.path.join(datasetsPath, "reducedSpotifyCharts.csv")
genresPath = os.path.join(datasetsPath, "genres.csv")
marketsPath = os.path.join(datasetsPath, "markets.csv")
tracksPath = os.path.join(datasetsPath, "tracks.csv")
albumsPath = os.path.join(datasetsPath, "albums.csv")
artistsPath = os.path.join(datasetsPath, "artists.csv")
peoplePath = os.path.join(datasetsPath, "people.csv")

# Countries
countriesPath = os.path.join(datasetsPath, "countries.csv")
altCountriesPath = os.path.join(datasetsPath, "altCountries.csv")

### Connection to Neo4j

In [3]:
# Neo4J params class
class Neo4jParams:
  def __init__(self, user, psw,dbname,db_psw,uri):
    self.user = user
    self.psw = psw
    self.dbname = dbname
    self.dbpsw = dbpsw
    self.uri = uri

In [4]:
#DB parameters
user="neo4j"
psw="neo4j"
dbname="SpotifyDB"
dbpsw="SpotifyDB"
uri = "bolt://localhost:7687"

params = Neo4jParams(user,psw,dbname,dbpsw,uri)

In [5]:
from neo4j import GraphDatabase

# test class

class Driver:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]


if __name__ == "__main__":
    greeter = Driver("bolt://localhost:7687", "neo4j", "SpotifyDB")
    greeter.print_greeting("hello, world")
    greeter.close()

hello, world, from node 38445


## Data ingestion

### Genres

In [6]:
# Load the CSV files in memory
genres = pd.read_csv(genresPath, sep=",", index_col="genre")

In [7]:
def createGenreID(genre):
    # Replace all special chars with "-"
    genreID = ""
    for char in genre:
        genreID += char if char.isalnum() else "-"
        
    return genreID

In [8]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

# To count how many nodes are created
totNodesCreated = 0

# Iterate over the album DataFrame
for genre, row in genres.iterrows():
    
    # Create genre ID from name
    genreID = createGenreID(genre)

    result = session.run("MERGE (g:Genre {id: $genreID}) RETURN g ", genreID = genreID)

    totNodesCreated += result.consume().counters.nodes_created

print(totNodesCreated)
session.close()
driver.close()

1617


### Countries

In [9]:
# Load the CSV files in memory
genres = pd.read_csv(countriesPath, sep=",",index_col="Code")

In [10]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

# To count how many nodes are created
totNodesCreated = 0

# Iterate over the album DataFrame
for countryCode, row in genres.iterrows():
    
    result = session.run("MERGE (c:Country {code: $countryCode, name: $countryName}) RETURN c ", 
    countryCode = countryCode,
    countryName = row['Name'])

    totNodesCreated += result.consume().counters.nodes_created

print(totNodesCreated)
session.close()
driver.close()

249


### Artists

In [11]:
# Load the CSV files in memory
artists = pd.read_csv(artistsPath, sep=",", index_col="id")

In [12]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

# To count how many nodes are created
totNodesCreated = 0

# Iterate over the album DataFrame
for artistID, row in artists.iterrows():
    
    # Retrieve the name of the artist
    artistName = row["name"]
    # Retrieve the popularity of the artist
    artistPopularity = row["popularity"]

    result = session.run("MERGE (a:Artist {id: $artistID, name: $artistName, popularity: $artistPopularity}) RETURN a ", 
                        artistID = artistID,
                        artistName = artistName,
                        artistPopularity = artistPopularity)

    totNodesCreated += result.consume().counters.nodes_created

    # Load genres as array
    genres = row["genres"].split(",") if not pd.isnull(row["genres"]) else []

    for genre in genres:
        session.run(""" MATCH (a:Artist {id: $artistID}) , (g:Genre {id: $genreID})
                        MERGE (a)-[r:hasGenre]->(g) RETURN a """, 
                        artistID = artistID,
                        genreID = createGenreID(genre))
         

print(totNodesCreated)
session.close()
driver.close()

8400


### Albums

In [13]:
# Load the CSV files in memory
albums = pd.read_csv(albumsPath, sep=",", index_col="id")

In [14]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

# To count how many nodes are created
totNodesCreated = 0

# Iterate over the album DataFrame
for albumID, row in albums.iterrows():

    # Retrieve the name of the album
    albumName = row["title"]
    
    # Retrieve the total tracks of the album
    albumTotalTracks = row["total_tracks"]

    # Manage release date taking into account release precision
    albumReleaseDate = row["release_date"]
    if(row["release_date_precision"]=="year"):
        albumReleaseDate += "-01-01"
    elif(row["release_date_precision"]=="month"):
        albumReleaseDate += "-01"
    
    # Retrieve album type
    albumType = row["album_type"]
    
    result = session.run("MERGE (a:Album {id: $albumID, name: $albumName, totalTracks: $albumTotalTracks, releaseDate: $albumReleaseDate, albumType: $albumType}) RETURN a ", 
                        albumID = albumID,
                        albumName = albumName,
                        albumTotalTracks = albumTotalTracks,
                        albumReleaseDate = albumReleaseDate,
                        albumType = albumType)

    totNodesCreated += result.consume().counters.nodes_created

    if(totNodesCreated%1000 == 0):
        print("💾 [CREATING] : "+str(totNodesCreated))

    # Load countries as array
    countries = row["available_countries"].split(",") if not pd.isnull(row["available_countries"]) else []

    for country in countries:
        session.run(""" MATCH (alb:Album {id: $albumID}), (c:Country {id: $countryCode} ) 
                        MERGE (alb)-[r:isAvailableIn]->(c) RETURN alb """, 
                        albumID = albumID,
                        countryCode = country)    
    
    # Load artists as array
    artists = row["artists"].split(",")
        
    for artistID in artists:
        session.run(""" MATCH (alb:Album {id: $albumID}), (a:Artist {id: $artistID})
                        MERGE (a)-[r:partecipateIn]->(alb) RETURN alb """, 
                        albumID = albumID,
                        artistID = artistID) 
        


print(totNodesCreated)
session.close()
driver.close()

17762


### Tracks

In [15]:
# Load the CSV files in memory
tracks = pd.read_csv(tracksPath, sep=",", index_col="id")

In [16]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

for trackID, row in tracks.iterrows():

    trackName = row["title"]

    # Retrieve all the technical charateristics
    duration = row["duration"]
    popularity = row["popularity"]
    explicit = row["explicit"] 
    key = row["key"] 
    tempo = row["tempo"] 
    mode = row["mode"]
    time_signature = row["time_signature"]
    acousticness = row["acousticness"] 
    danceability = row["danceability"] 
    energy = row["energy"] 
    loudness = row["loudness"] 
    liveness = row["liveness"] 
    valence = row["valence"] 
    speechiness = row["speechiness"] 
    instrumentalness = row["instrumentalness"] 

    result = session.run("""MERGE (t:Track {id: $trackID,
                                            name: $trackName,
                                            duration: $duration, 
                                            popularity: $popularity, 
                                            explicit: $explicit,
                                            key: $key,
                                            tempo: $tempo,
                                            mode: $mode,
                                            time_signature: $time_signature,
                                            acousticness: $acousticness,
                                            danceability: $danceability,
                                            energy: $energy,
                                            loudness: $loudness,
                                            liveness: $liveness,
                                            valence: $valence,
                                            speechiness: $speechiness,
                                            instrumentalness: $instrumentalness}) 
                            RETURN t """, 
                            trackID = trackID,
                            trackName = trackName,
                            duration = duration, 
                            popularity = popularity, 
                            explicit = explicit,
                            key = key,
                            tempo = tempo,
                            mode = mode,
                            time_signature = time_signature,
                            acousticness = acousticness,
                            danceability = danceability,
                            energy = energy,
                            loudness = loudness,
                            liveness = liveness,
                            valence = valence,
                            speechiness = speechiness,
                            instrumentalness = instrumentalness                        
                            )

    totNodesCreated += result.consume().counters.nodes_created

    if(totNodesCreated%1000):
        print("💾 [CREATING] : " + str(totNodesCreated))

    #Retrieve albumID
    albumID = row["album"]
    session.run(""" MATCH (t:Track {id: $trackID}} ),(alb:Album {id: $albumID})
                    MERGE (t)-[r:isPartOf]->(alb) RETURN t """, 
                    trackID = trackID,
                    albumID = albumID)    

    # Load countries as array
    countries = row["available_countries"].split(",") if not pd.isnull(row["available_countries"]) else []

    for country in countries:
        session.run(""" MATCH (t:Track {id: $trackID}} ), (c:Country {id: $countryCode} ) 
                        MERGE (t)-[r:isAvailableIn]->(c) RETURN t """, 
                        trackID = trackID,
                        countryCode = country)    
    
    # Load artists as array
    artists = row["artists"].split(",")
        
    for artistID in artists:
        session.run(""" MATCH (a:Artist {id: $artistID}), (t:Track {id: $trackID}} )
                        MERGE (a)-[r:partecipateIn]->(t) RETURN t """, 
                        artistID = artistID,
                        trackID = trackID)

print(totNodesCreated)
session.close()
driver.close()

TypeError: can only concatenate str (not "int") to str

### Charts

In [ ]:
def loadCountries():
    countries = pd.read_csv(countriesPath, sep=",")
    altCountries = pd.read_csv(altCountriesPath, sep=",")
    altCountries.columns = ["AlternativeName", "Name"]
    
    return countries, altCountries


def getCountryCode(countryName, countries, altCountries):
    # Try to retrieve ISO CODE of the country
    try:
        matchedCountries = countries[countries["Name"].str.contains(countryName)]
        countryCode = matchedCountries["Code"].iloc[0]
    except IndexError as e:
        # Look if an alternative name was used
        alternativeMatchedCountries = altCountries[altCountries["AlternativeName"].str.contains(countryName)]
        countryName = alternativeMatchedCountries["Name"].iloc[0]

        matchedCountries = countries[countries["Name"] == countryName]
        countryCode = matchedCountries["Code"].iloc[0]
    
    return countryCode, countryName

In [ ]:
# Load countries dataframe
countries, altCountries = loadCountries()

# Load the CSV files in memory
charts = pd.read_csv(spotifyChartsPath , sep=",")

# Aggregate the original dataframe to identify a specific chart using COUNTRY and DATE
chartsDF = charts.groupby(
    ["country", "date"]).size().reset_index(name="total_tracks")

# Removing global
chartsDF = chartsDF.drop(index=chartsDF[chartsDF["country"] == "Global"].index)
charts = charts.drop(index=charts[charts["country"] == "Global"].index)

totalRows = len(charts.index)

In [ ]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

#I iterate through the dataframe
for index, row in chartsDF.iterrows():

    # Retrieve country and date
    countryName = row["country"]
    chartDate = row["date"]
    topNumType = 100
    chartTotalTracks = row["total_tracks"]

    # Reformat date
    chartDate = datetime.datetime.strptime(chartDate, "%d/%m/%Y").strftime("%Y-%m-%d")

    # Get the country code
    countryCode, _ = getCountryCode(countryName, countries, altCountries)

    # Create a uniqueID
    chartID = "top-{}-{}-{}".format(topNumType, countryCode, chartDate)

    # Add the name of the Chart
    chartName = "TOP {} {}".format(topNumType, countryName)

    chartType = "top"
  
    result = session.run("MERGE (ch:Chart {id: $chartID, name: $chartName, totalTracks: $chartTotalTracks, date: $chartDate, chartType: $chartType}) RETURN ch ", 
                        chartID = chartID,
                        chartName = chartName,
                        chartTotalTracks = chartTotalTracks,
                        chartDate = chartDate,
                        chartType = chartType)

    totNodesCreated += result.consume().counters.nodes_created

    if(totNodesCreated%1000 == 0):
        print("💾 [CREATING] : " + str(totNodesCreated))
    
    #Add country
    session.run(""" MATCH (ch:Chart {id: $chartID}} ), (c:Country {id: $countryCode} ) 
                        MERGE (ch)-[r:isReferredTo]->(c) RETURN ch """, 
                        chartID = chartID,
                        countryCode = countryCode) 

print(totNodesCreated)
session.close()
driver.close()

In [ ]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

for index, row in charts.iterrows():

    # Retrieve the position of track
    position = row["position"]

    # Get the track id from the uri
    trackID = row['uri'].removeprefix("https://open.spotify.com/track/")

    # Retrieve country and date
    countryName = row["country"]
    date = row["date"]

    # Reformat date
    date = datetime.datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d")

    # Get the country code
    countryCode, _ = getCountryCode(countryName, countries, altCountries)

    # Create a uniqueID
    chartID = "top-100-{}-{}".format(countryCode, date)

    #Add country
    session.run(""" MATCH (t:Track {id: $trackID} ), (ch:Chart {id: $chartID}})
                    MERGE (t)-[r:isPositionedIn {position: $position}]->(ch) RETURN t""", 
                    trackID = trackID,
                    chartID = chartID,
                    position = position) 

print(totNodesCreated)
session.close()
driver.close()

## People

In [ ]:
# Load the CSV files in memory
people = pd.read_csv(peoplePath, sep=",", index_col="id")

In [ ]:
def autoCompleteDate(dateStr):
    if dateStr is not None:
        if len(dateStr) == 4:
            dateStr += "-01-01"
        elif len(dateStr) == 7:
            dateStr += "-01"

    return dateStr

In [ ]:
# connect to the DB
driver = GraphDatabase.driver(params.uri, auth=(params.user, params.dbpsw))
# create a session
session = driver.session()

groupedPeople = people.groupby(["id"])

index = 0

# iterate over each group
for peopleID, peopleGroup in groupedPeople:

    name = peopleGroup.iloc[0]["name"] if not pd.isnull(
        peopleGroup.iloc[0]["name"]) else ""
    surname = peopleGroup.iloc[0]["surname"] if not pd.isnull(
        peopleGroup.iloc[0]["surname"]) else ""
    birthDate= peopleGroup.iloc[0]["birthdate"] if not pd.isnull(
        peopleGroup.iloc[0]["birthdate"]) else ""
    deathDate= peopleGroup.iloc[0]["deathdate"] if not pd.isnull(
        peopleGroup.iloc[0]["deathdate"]) else ""
    nationality= peopleGroup.iloc[0]["nationality"] if not pd.isnull(
        peopleGroup.iloc[0]["nationality"]) else ""

    # Get the list of artists in which the person appear
    artists = []
    for _, row in peopleGroup.iterrows():
        artists.append(row["artist"])

    # Create a uniqueID
    peopleID = "people-{}".format(index)
    index += 1

    # Manage dates
    birthDate = autoCompleteDate(birthDate)
    birthDate = birthDate if not birthDate is None else ""
    
    deathDate = autoCompleteDate(deathDate)
    deathDate = deathDate if not deathDate is None else ""


    result = session.run("MERGE (p:Person {id: $personID, name: $personName, surname: $personSurname, birthDate: $birthDate, deathDate: $deathDate}) RETURN p ", 
                        personID = peopleID,
                        personName = name,
                        personSurname = surname,
                        birthDate = birthDate,
                        deathDate = deathDate)

    totNodesCreated += result.consume().counters.nodes_created

    # Add nationality
    # Create the RDF node
    if nationality is not None:
        #Add country
        session.run(""" MATCH (p:Person {id: personID} ), (c:Country {id: $countryCode}})
                    MERGE (p)-[r:hasNationality]->(c) RETURN p""", 
                    personID = peopleID,
                    countryCode = nationality)

    #manage edge connectig artists and people
    for artistID in artists:
        session.run(""" MATCH (p:Person {id: $personID}}), (a:Artist {id: $artistID})
                    MERGE (p)-[r:isMemberOf]->(a) RETURN p """, 
                    personID = peopleID,
                    artistID = artistID)


print(totNodesCreated)
session.close()
driver.close()